### Building a Tree Code VII: Colliding Galaxies

In [ ]:
%matplotlib qt

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
from treeclasses import ParticleSet, BHTree, Hamiltonian, State, KDK, Conservation
from timerclass import Timer

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plt.rcParams['figure.dpi'] = 150

As an illustrative problem, we'll reproduce a galaxy collision from Dubinski, Mihos, and Hernquist "Using Tidal Tails to Probe Dark Matter Halos", 1996 ApJ 462 p. 567. This is their model B. The data containing the initial conditions is in the text file ```dubinski.tab```. 

This was originally run in 1995 on 16 processors of a Cray T3D supercomputer. The particular machine used had 32 processors, a 150 MHz clock, & 8 GB of RAM. My laptop in 20201 has a clock of 2.3 Ghz, 32 GB of ram, and 16 processors!

In [ ]:
with open("dubinski.tab") as f:
    data = np.loadtxt(f)
print(data.shape)

In [ ]:
# create a particle set
PS = ParticleSet()

# allocate storage
N = data.shape[0]
PS.reserve(N)

# copy in the data
PS.mass[:] = data[:,0]
PS.r[:,:]  = data[:,1:4]
PS.v[:,:]  = data[:,4:]

# the data consists of sets of particles for the disks, bulges, and halos in each galaxy
Ndisk = 16384
Nbulge = 8192
Nhalo = 16384
Ngal = Ndisk+Nbulge+Nhalo
assert( Ngal*2 == N )

Create numpy ranges to use for plotting the two galaxies in different colors. We'll plot only the bulges and disks and leave out the halo particles from the plot. Since the halo particles are really intended to represent the dark matter halo, this gives us an animation which shows the stars (as well as matplotlib can do so!).

In [ ]:
disk1 = np.s_[:Ndisk+Nbulge]
disk2 = np.s_[Ngal:Ngal+Ndisk+Nbulge]

Create a 3D plot to animate using matplotlib:

In [ ]:
PS.computeBoundingBox()
centre, halfWidth = PS.getBoundingBox()

fig = plt.figure()
ax = plt.axes(projection='3d')
c, hw = PS.getBoundingBox()
ax.set_xlim(c[0]-hw,c[0]+hw)
ax.set_ylim(c[0]-hw,c[0]+hw)
ax.set_zlim(c[0]-hw,c[0]+hw)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

plot1, = ax.plot3D(PS.r[disk1,0], PS.r[disk1,1], PS.r[disk1,2], 'b.', alpha=0.05, markersize=0.5)
plot2, = ax.plot3D(PS.r[disk2,0], PS.r[disk2,1], PS.r[disk2,2], 'r.', alpha=0.05, markersize=0.5)

steptxt = ax.text2D(0.2,1.0, f"step:    0   time: {0.0:8.2e}\nerror: E: {0:.2e}  P: {0:.2e}", transform=ax.transAxes)

and a function to update the animation:

In [ ]:
C = Conservation(PS)

def updatePlot(S, C, timer):
    ps = S.ps
    plot1.set_data(ps.r[disk1,0], ps.r[disk1,1]);  plot1.set_3d_properties(ps.r[disk1,2])
    plot2.set_data(ps.r[disk2,0], ps.r[disk2,1]);  plot2.set_3d_properties(ps.r[disk2,2])
    
    K, P, E = C.getEnergy()
    # relative error in energy
    DErel = (K+P-C.E0)/C.E0
    
    line = f"step: {S.step:4d}   time: {S.time:8.2e}   $\Delta E$: {DErel:.2e}\n"
    line += f"rate: {ps.N/timer.elapsed():.2e}"
    steptxt.set_text(line)
    
    # this updates the animation w/o stealing the window focus
    plt.gcf().canvas.draw_idle(); plt.gcf().canvas.start_event_loop(0.001)

Set up the system parameters:

```
maxLeafSize: tree parameter
  maxSrcLen: space to reserve for interaction lists
      theta: Barnes-Hut accuracy parameter
  epsSmooth: Smoothing (softening) parameter for gravity
         dt: timestep
     nsteps: number of timesteps
       skip: number of steps to skip between animation frames
```

Instantiate the Hamiltonian and State classes, and take an initial zero-timestep step so
that the potential of the initial conditions is calculated:

In [ ]:
maxLeafSize = 16
maxSrcLen = PS.N
theta = 0.7
epsSmooth = 0.025
dt = 0.1
skip = 1

H = Hamiltonian(theta, epsSmooth, maxLeafSize, maxSrcLen)
S = State(PS, H)

S = KDK(0.0, H, S) # take a step w/ dt=0 just to get initial potential
C = Conservation(PS)

Finally, we can run our simulation!

In [ ]:
def runit(S):

    timer = Timer()

    while(S.time<100.0):

        timer.cstart()
        S = KDK(dt, H, S)
        timer.stop()
    
        if S.step%skip==0:
            updatePlot(S, C, timer)      

    updatePlot(S, C, timer)

In [ ]:
runit(S)